In [4]:
import arcpy
arcpy.env.workspace = r'C:\Users\ltmsbi\Documents\ArcGIS\Projects\Final_Deployment\Final_Deployment.gdb'

In [8]:
# Create layers per-species and geometry then add names to list for spatial joining


# Create list of n + n*3 files to process, where n is the number of species
presences = [] # empty list to append items onto

# Create dictionary of long names
# Values are not used in this code and are only present for human readability
species_fullnames = {
    "phrag": "Phragmites, Unspecified",
    "knot": "Knotweed, Unspecified",
    "wp": "Wild Parsnip",
    "toh": "Tree-of-Heaven (Ailanthus)",
    "pl": "Purple Loosestrife"
}

# Extract only the keys to a list
species_shortnames = list(species_fullnames.keys())


# Add model data layer names to processing list ----------


# Define the threshold used to determine presences
threshold = 'precision'

# Add names to list
for n in range(0,len(species_shortnames)):
    # Add n items for model data 
    presences.append('SVI_Project_presences_'+species_shortnames[n]+"_"+threshold)
    # Add n*3 items for iMap data (accounts for 3 geometry types) 

    
# ----------
    
# Separate out iMap records by species and geometry ----------


# IDs that iMap assigns to the various species of interest
# Used for building queries to select by attribute
jurisdiction_ids = {
    "phrag": 1277,
    "wp": 1182,
    "pl": 1265,
    "toh": 1167,
    "knot": (1074, 1191, 1278, 1479) # Includes Japanese knotweed, giant knotweed, bohemian knotweed, and knotweed species unknown
}

# Define list of geometries in the iMap data
imap_geometries = ["POINT", "LINE", "POLYGON"]

for n in range(0,len(species_shortnames)):
    # Set up per-species query for selecting by attribute
    if species_shortnames[n] is "knot":
        # Set initial SQL query
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids["knot"][0])
        # Add more conditions to query
        for ID in jurisdiction_ids["knot"][1:]:
            idClause = idClause + " Or jurisdiction_species_id = " + str(ID)
    else:
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids[species_shortnames[n]]) 
    
    # Copy a per-species subset for each geometry type
    for k in range(0,len(imap_geometries)):
        sel = arcpy.management.SelectLayerByAttribute("PRESENCE_"+imap_geometries[k], "NEW_SELECTION", idClause)
        imap_presence = 'PRESENCE_'+imap_geometries[k]+"_"+species_shortnames[n]
        arcpy.management.CopyFeatures(sel, imap_presence)
        presences.append(imap_presence)
        # Remove variables and selections to save memory
        del sel, imap_presence
        arcpy.management.SelectLayerByAttribute("PRESENCE_"+imap_geometries[k], "CLEAR_SELECTION")
        
        
# ----------

print("Number of layers to process: "+str(len(presences)))
print(presences)

Number of layers to process: 20
['SVI_Project_presences_phrag_precision', 'SVI_Project_presences_knot_precision', 'SVI_Project_presences_wp_precision', 'SVI_Project_presences_toh_precision', 'SVI_Project_presences_pl_precision', 'PRESENCE_POINT_phrag', 'PRESENCE_LINE_phrag', 'PRESENCE_POLYGON_phrag', 'PRESENCE_POINT_knot', 'PRESENCE_LINE_knot', 'PRESENCE_POLYGON_knot', 'PRESENCE_POINT_wp', 'PRESENCE_LINE_wp', 'PRESENCE_POLYGON_wp', 'PRESENCE_POINT_toh', 'PRESENCE_LINE_toh', 'PRESENCE_POLYGON_toh', 'PRESENCE_POINT_pl', 'PRESENCE_LINE_pl', 'PRESENCE_POLYGON_pl']


In [ ]:
# Spatial join repeatedly

# Effectively, for iMap and model data, each species, and geometry/record type (if iMap data):
    # Join number of points to the reporting grid created in previous run
    
    
    # Calculate model "not detected" as all model points joined - model species presence

In [ ]:
# Build the analysis-ready version (1 layer with numbers for all species)

# Build the viewing-ready version (n per-species layers combined into one)

# For each species:
    # Select records with non-null values for that species
    # Add & calculate common name, jurisdiction id, and op criteria fields
# Merge layers into one for upload into AGOL